# 1st

## Model Explore

In [ ]:
!pip install -q transformers accelerate bitsandbytes huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 16.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
notebook_login()

## more model

In [ ]:
torch.cuda.is_available()

True

In [ ]:
model_id = "meta-llama/Llama-2-7b-hf"

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Load model with 8-bit precision (needs less memory)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_8bit=True,  # For lower RAM usage (needs `bitsandbytes`)
    torch_dtype=torch.float16,
)


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def answer(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    # output = model.generate(**inputs, max_new_tokens=200)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        eos_token_id=tokenizer.eos_token_id)
    # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
prompt = " I have knee pain which medicine should i take?"
answer(prompt)

' I have knee pain which medicine should i take?\nHow long will it take for the pain to go away?\nI have a severe headache what should I do?\nI have a cold, what should I do?\nI have a fever, what should I do?\nI have a cough, what should I do?\nI have a sore throat, what should I do?\nI have a runny nose, what should I do?\nI have a rash, what should I do?\nI have a red eye, what should I do?\nI have a stomachache, what should I do?\nI have a sore throat, what should I do?\nI have a headache, what should I do'

In [ ]:
prompt = "I have headache which medicine should i take?"
answer(prompt)

'I have headache which medicine should i take?\nHow do I know if my headache is serious?\nWhat are the symptoms of headache?\nWhat is the best medicine for headache?\nWhat is the best medicine for headache?\nWhat are the symptoms of a headache?\nHow do I know if my headache is serious?\nHow do you know if your headache is serious?\nWhat are the symptoms of a headache?\nWhat is the best medicine for headache?\nWhat is the best medicine for headache?\nWhat is the best medicine for headache?\nWhat is the best medicine for headache?\nWhat is the best medicine for headache?\nWhat is the best medicine for headache?\nWhat'

# detection

In [ ]:
# install embedding library
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer, util
import torch

In [ ]:


#  10 questions and gold answers
data = [
    {"question": "What is the chemical symbol for water?",
     "standard_answer": "H₂O"},
    {"question": "Which planet is known as the Red Planet?",
     "standard_answer": "Mars"},
    {"question": "What is the capital city of France?",
     "standard_answer": "Paris"},
    {"question": "How many continents are there on Earth?",
     "standard_answer": "Seven (Africa, Antarctica, Asia, Europe, North America, Oceania, South America)"},
    {"question": "Who wrote the play “Romeo and Juliet”?",
     "standard_answer": "William Shakespeare"},
    {"question": "What is the largest mammal in the world?",
     "standard_answer": "The blue whale (Balaenoptera musculus)"},
    {"question": "In what year did the Titanic sink?",
     "standard_answer": "1912"},
    {"question": "What is the value of π (pi) to two decimal places?",
     "standard_answer": "3.14"},
    {"question": "Which element has the atomic number 6?",
     "standard_answer": "Carbon"},
    {"question": "What language is primarily spoken in Brazil?",
     "standard_answer": "Portuguese"},
]

# 2. Load SBERT embedder onto GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedder = SentenceTransformer("all-MiniLM-L6-v2").to(device)




In [ ]:
# --- 3. Define sampling settings (seed + temperature) ---
N = 5
seeds = [42, 43, 44, 45, 46]
temps = [0.5, 0.6, 0.7, 0.8, 0.9]
settings = list(zip(seeds, temps))
all_avg_scores = []
# --- 4. Loop through each Q&A pair ---
for item in data:
    q  = item["question"]
    ga = item["standard_answer"]

    # 4a. Generate N answers with different seeds & temperatures
    samples = []
    for seed, temp in settings:
        torch.manual_seed(seed)
        prompt_text = f"Question: {q}\nAnswer:"
        inputs = tokenizer(prompt_text, return_tensors="pt").to(device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=temp,
            top_p=0.9,
            top_k=50,
            eos_token_id=tokenizer.eos_token_id
        )
        ans = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        samples.append(ans)

    # 4b. Embed the gold answer and the samples
    gold_emb   = embedder.encode(ga,       convert_to_tensor=True).to(device)
    sample_emb = embedder.encode(samples,  convert_to_tensor=True).to(device)

    # 4c. Compute cosine similarities and average
    sims = util.cos_sim(gold_emb, sample_emb)[0]
    avg_score = sims.mean().item()
    all_avg_scores.append(avg_score)

    # 4d. Print the results
    print("-" * 80)
    print(f"Q: {q}")
    print(f"Gold: {ga!r}")
    print(f"→ Average semantic similarity: {avg_score:.3f}")
    for i, (score, ans) in enumerate(zip(sims, samples), 1):
        print(f"   {i:>2}. [{temps[i-1]:.1f} temp, seed={seeds[i-1]}] {score.item():.3f} → {ans}")
    print("-" * 80)

--------------------------------------------------------------------------------
Q: What is the chemical symbol for water?
Gold: 'H₂O'
→ Average semantic similarity: 0.135
    1. [0.5 temp, seed=42] 0.120 → Question: What is the chemical symbol for water?
Answer: The chemical symbol for water is H2O.
Question: What is the chemical symbol for hydrogen?
Answer: The chemical symbol for hydrogen is H.
Question: What is the chemical symbol for oxygen?
Answer: The chemical symbol for oxygen is O.
Question: What is the chemical symbol for nitrogen?
Answer: The chemical symbol for nitrogen is N.
Question: What is the chemical symbol for chlorine?
Answer: The chemical symbol for chlorine is Cl.
Question: What is the chemical symbol for sodium?
Answer: The chemical symbol for sodium is Na.
Question: What is the chemical symbol for potassium?
Answer:
    2. [0.6 temp, seed=43] 0.150 → Question: What is the chemical symbol for water?
Answer: H2O
Question: What is the chemical symbol for hydrogen?


In [ ]:

overall_avg = sum(all_avg_scores) / len(all_avg_scores)
print(f"\n=== Overall Average Across {len(all_avg_scores)} Questions: {overall_avg:.3f} ===")


=== Overall Average Across 10 Questions: 0.346 ===
